In [1]:
#!pip3 install opencv-python

In [2]:
import cv2
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np 

model=load_model('')

- **Quantizzo modello**

In [3]:
'''
# Converti il modello in un modello TensorFlow Lite quantizzato
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
quantized_tflite_model = converter.convert()

# Salva il modello quantizzato su disco
with open('/users/emanueleamato/Downloads/Esame Echo Emanuele /20240215/model_quantized.tflite', 'wb') as f:
    f.write(quantized_tflite_model)
'''

"\n# Converti il modello in un modello TensorFlow Lite quantizzato\nconverter = tf.lite.TFLiteConverter.from_keras_model(model)\nconverter.optimizations = [tf.lite.Optimize.DEFAULT]\nquantized_tflite_model = converter.convert()\n\n# Salva il modello quantizzato su disco\nwith open('/users/emanueleamato/Downloads/Esame Echo Emanuele /20240215/model_quantized.tflite', 'wb') as f:\n    f.write(quantized_tflite_model)\n"

- **Real time Inference** 

In [8]:
import cv2
import numpy as np
import tensorflow as tf

# Carica il modello TensorFlow Lite
model = tf.lite.Interpreter(model_path="model_quantized.tflite")
model.allocate_tensors()

# Definisci una funzione per preprocessare i frame
def preprocess_frame(frame):
    resized_frame = cv2.resize(frame, (224, 224))
    normalized_frame = resized_frame / 255.0
    preprocessed_frame = np.expand_dims(normalized_frame.astype(np.float32), axis=0) 
    return preprocessed_frame



# Apri la sorgente video (ad esempio, una duplicazione dello schermo)
video_capture = cv2.VideoCapture(0)  # Modifica il numero per utilizzare una sorgente video diversa

while True:
    # Leggi il frame dalla sorgente video
    ret, frame = video_capture.read()
    
    if not ret:
        break

    # Preprocessa il frame
    processed_frame = preprocess_frame(frame)

    # Esegui l'inferenza sul frame
    model.set_tensor(model.get_input_details()[0]['index'], processed_frame)
    model.invoke()

    # Ottieni l'output dell'inferenza
    output_details = model.get_output_details()[0]
    output_data = model.get_tensor(output_details['index'])

    # Trova la classe predetta e la probabilità massima
    predicted_class_index = np.argmax(output_data)
    labels = ["2CH", "4CH"]  # Sostituisci con le tue etichette di classe
    predicted_class_name = labels[predicted_class_index]
    max_probability = np.max(output_data)

    # Disegna il testo sul frame con il nome della classe predetta e la probabilità
    text = f"Class: {predicted_class_name}, Probability: {max_probability}"
    cv2.putText(frame, text, (25, 25), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
    
    # Mostra il frame con le previsioni
    cv2.imshow('Frame', frame)
    
    # Esci se viene premuto il tasto 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Rilascia la cattura del video e chiudi le finestre
video_capture.release()
cv2.destroyAllWindows()


In [7]:
'''
import pyautogui 
import cv2
import numpy as np
import tensorflow as tf

# Carica il modello TensorFlow Lite
model = tf.lite.Interpreter(model_path="model_quantized.tflite")
model.allocate_tensors()

# Definisci una funzione per preprocessare i frame
def preprocess_frame(frame):
    resized_frame = cv2.resize(frame, (256, 256))
    normalized_frame = resized_frame / 255.0
    preprocessed_frame = np.expand_dims(normalized_frame.astype(np.float32), axis=0) 
    return preprocessed_frame


while True:
    # Leggi il frame dalla sorgente video
    screenshot=pyautogui.screenshot(region=(1920,1080,1920,1080))
    frame=np.array(screenshot)
    preprocessed_frame=preprocess_frame(frame)

    input_details=model.get_input_details()
    output_details=model.get_output_details()
    model.set_tensor(input_details[0]['index'],preprocessed_frame)
    model.invoke()

    output_data=model.get_tensor(output_details[0]['index'])
    predicted_class_index = np.argmax(output_data)
    labels = ["Good", "Medium", "Poor"]  # Sostituisci con le tue etichette di classe
    predicted_class_name = labels[predicted_class_index]
    max_probability = np.max(output_data)

    # Disegna il testo sul frame con il nome della classe predetta e la probabilità
    text = f"Class: {predicted_class_name}, Probability: {max_probability}"
    cv2.putText(frame, text, (25, 25), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
    
    # Esci se viene premuto il tasto 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cv2.destroyAllWindows()
'''

'\nimport pyautogui \nimport cv2\nimport numpy as np\nimport tensorflow as tf\n\n# Carica il modello TensorFlow Lite\nmodel = tf.lite.Interpreter(model_path="model_quantized.tflite")\nmodel.allocate_tensors()\n\n# Definisci una funzione per preprocessare i frame\ndef preprocess_frame(frame):\n    resized_frame = cv2.resize(frame, (256, 256))\n    normalized_frame = resized_frame / 255.0\n    preprocessed_frame = np.expand_dims(normalized_frame.astype(np.float32), axis=0) \n    return preprocessed_frame\n\n\nwhile True:\n    # Leggi il frame dalla sorgente video\n    screenshot=pyautogui.screenshot(region=(1920,1080,1920,1080))\n    frame=np.array(screenshot)\n    preprocessed_frame=preprocess_frame(frame)\n\n    input_details=model.get_input_details()\n    output_details=model.get_output_details()\n    model.set_tensor(input_details[0][\'index\'],preprocessed_frame)\n    model.invoke()\n\n    output_data=model.get_tensor(output_details[0][\'index\'])\n    predicted_class_index = np.arg